# Native Code and SIMD 

Julia can generate code for a variety of architectures. What's even nicer is that Julia has these nice code introspection tools that let you peek into the different stages of the compilation process, all the way down to generation of machine code. 


Consider integer addition.

In [ ]:
1 + 1

The Julia macro `@code_native` when placed before a valid expression, will display the machine code generated as a result of compiling that expression.

In [ ]:
@code_native 1 + 1

Now check the native code generated when adding two double precision floating point numbers.

In [ ]:
@code_native 1. + 1.

Let's move to another example. Consider a simple function that loops over array elements and adds them to an accumulator.

In [ ]:
function mysum(a::Vector)
    total = zero(eltype(a))
    for x in a
        total += x
    end
    return total
end

Now let's create an array to loop over:

In [ ]:
a = rand(10^8);

And load our trusty benchmarking tools:

In [ ]:
using Compat
using BenchmarkTools

In [ ]:
@btime mysum(a)

But what kind of native code did it generate?

In [ ]:
@code_native mysum(a)

## `@simd` - generate vectorized code

Now let us introduce another common macro in Julia: the `@simd`, which is designed to take advantage of SIMD vector lanes of the processor: 

In [ ]:
function mysum_vectorized(a::Vector)
    total = zero(eltype(a))
    @simd for x in a
        total += x
    end
    return total
end

Is it faster?

In [ ]:
@btime mysum_vectorized(a)

In [ ]:
@code_native mysum_vectorized(a)

Do you see the difference between the code generated?